In [1]:
import tensorflow as tf
import numpy as np
import DataSets as ds
import Layers

/Users/auredt7892/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def get_dict(database,IsTrainingMode):
	xs,ys = database.NextTrainingBatch()
	return {x:xs,y_desired:ys,ITM:IsTrainingMode}

In [3]:
LoadModel = False
KeepProb_Dropout = 0.9

experiment_name = '10k_Dr%.3f'%KeepProb_Dropout
#train = ds.DataSet('../DataBases/data_1k.bin','../DataBases/gender_1k.bin',1000)
train = ds.DataSet('../DataBases/data_10k.bin','../DataBases/gender_10k.bin',10000)
#train = ds.DataSet('../DataBases/data_100k.bin','../DataBases/gender_100k.bin',100000)
test = ds.DataSet('../DataBases/data_test10k.bin','../DataBases/gender_test10k.bin',10000)

nb data :  10000
nb data :  10000


In [4]:
with tf.name_scope('input'):
	x = tf.placeholder(tf.float32, [None, train.dim],name='x')
	y_desired = tf.placeholder(tf.float32, [None, 2],name='y_desired')
	ITM = tf.placeholder("bool", name='Is_Training_Mode')

with tf.name_scope('CNN'):
	t = Layers.unflat(x,48,48,1)
	nbfilter = 3
	for k in range(4):
		for i in range(2):
			t = Layers.conv(t,nbfilter,3,1,ITM,'conv_%d_%d'%(nbfilter,i),KeepProb_Dropout)
		t = Layers.maxpool(t,2,'pool')
		nbfilter *= 2
	
	t = Layers.flat(t)
	#t = Layers.fc(t,50,ITM,'fc_1',KeepProb_Dropout)
	y = Layers.fc(t,2,ITM,'fc_2',KP_dropout=1.0,act=tf.nn.log_softmax)

with tf.name_scope('cross_entropy'):
	diff = y_desired * y 
	with tf.name_scope('total'):
		cross_entropy = -tf.reduce_mean(diff)
	tf.summary.scalar('cross entropy', cross_entropy)	
	
with tf.name_scope('accuracy'):
	with tf.name_scope('correct_prediction'):
		correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_desired, 1))
	with tf.name_scope('accuracy'):
		accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
	tf.summary.scalar('accuracy', accuracy)	

with tf.name_scope('learning_rate'):
	global_step = tf.Variable(0, trainable=False)
	learning_rate = tf.train.exponential_decay(1e-3,global_step,1000, 0.75, staircase=True)

with tf.name_scope('learning_rate'):
    tf.summary.scalar('learning_rate', learning_rate)

#train_step = tf.train.GradientDescentOptimizer(0.00001).minimize(cross_entropy)
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy,global_step=global_step)
merged = tf.summary.merge_all()

Acc_Train = tf.placeholder("float", name='Acc_Train');
Acc_Test = tf.placeholder("float", name='Acc_Test');
MeanAcc_summary = tf.summary.merge([tf.summary.scalar('Acc_Train', Acc_Train),\
                                    tf.summary.scalar('Acc_Test', Acc_Test)])

unflat output   Tensor("CNN/Reshape:0", shape=(?, 48, 48, 1), dtype=float32)
conv_3_0 input   Tensor("CNN/Reshape:0", shape=(?, 48, 48, 1), dtype=float32)
conv_3_0 W   (3, 3, 1, 3)
conv_3_1 input   Tensor("CNN/conv_3_0/cond/Merge:0", shape=(?, 48, 48, 3), dtype=float32)
conv_3_1 W   (3, 3, 3, 3)
conv_6_0 input   Tensor("CNN/pool/MaxPool:0", shape=(?, 24, 24, 3), dtype=float32)
conv_6_0 W   (3, 3, 3, 6)
conv_6_1 input   Tensor("CNN/conv_6_0/cond/Merge:0", shape=(?, 24, 24, 6), dtype=float32)
conv_6_1 W   (3, 3, 6, 6)
conv_12_0 input   Tensor("CNN/pool_1/MaxPool:0", shape=(?, 12, 12, 6), dtype=float32)
conv_12_0 W   (3, 3, 6, 12)
conv_12_1 input   Tensor("CNN/conv_12_0/cond/Merge:0", shape=(?, 12, 12, 12), dtype=float32)
conv_12_1 W   (3, 3, 12, 12)
conv_24_0 input   Tensor("CNN/pool_2/MaxPool:0", shape=(?, 6, 6, 12), dtype=float32)
conv_24_0 W   (3, 3, 12, 24)
conv_24_1 input   Tensor("CNN/conv_24_0/cond/Merge:0", shape=(?, 6, 6, 24), dtype=float32)
conv_24_1 W   (3, 3, 24, 24)
flat out